**Черновые тесты с крестом**

---

Обычный TT-cross используется для оптимизации функций.

## Loading and importing modules

In [6]:
from IPython.display import clear_output
! pip install teneva==0.11.1
clear_output()

In [7]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Code

In [15]:
def solve(func, m):
    opt = {'i_max': None, 'y_max': None}

    def black_box(I):
        y = func.get_f_ind(I)
        s = np.argmax(y)
        if opt['y_max'] is None or abs(y[s]) > opt['y_max']:
            opt['i_max'] = I[s, :].copy()
            opt['x_max'] = teneva.ind_to_poi(opt['i_max'], func.a, func.b, func.n, func.kind)
            opt['y_max'] = abs(y[s])
        return y
    
    t = tpc()
    cache = {}
    Y = teneva.tensor_rand(func.n, r=1)
    Y = teneva.cross(black_box, Y, m, dr_min=0, dr_max=1, cache=cache, log=False)
    Y = teneva.truncate(Y, 1.E-10)

    # To check the approximation accuracy:
    func.t = tpc() - t
    func.method = 'CRO'
    func.prep(Y)
    I_trn, Y_trn = teneva.cache_to_data(cache)
    func.set_trn_ind(I=I_trn, Y=Y_trn)
    func.check()
    func.info() # Note error: train / test will be printed

    return Y, opt

## Test

In [16]:
d = 10        # Dimension
n = 64        # Grid size
m_trn = 1.E+4 # Train size
m_tst = 1.E+4 # Test size

In [17]:
for func in teneva.func_demo_all(d, only_with_min=True):
    func.clear()
    func.set_grid(n, kind='uni')
    func.build_tst_ind(m_tst)
    Y, opt = solve(func, m_trn)
    print(opt['y_max'], func.y_min)

Ackley          [CRO          ] > error: 3.8e-03 / 2.7e-02 | rank:  1.0 | time:   0.281
22.24598392302177 0.0
Alpine          [CRO          ] > error: 6.5e-02 / 4.5e-01 | rank:  1.0 | time:   0.204
86.7031202338489 0.0
Dixon           [CRO          ] > error: 1.1e-01 / 9.8e-01 | rank:  1.0 | time:   0.208
2381521.0 0.0
Exponential     [CRO          ] > error: 8.2e-16 / 8.4e-16 | rank:  1.0 | time:   0.216
0.6058433729196858 -1.0
Grienwank       [CRO          ] > error: 5.5e-02 / 4.8e-01 | rank:  1.0 | time:   0.334
900.9999607771135 0.0
Michalewicz     [CRO          ] > error: 9.7e-02 / 1.9e+00 | rank:  1.0 | time:   0.209
8.01426235826608 -9.66015
Qing            [CRO          ] > error: 1.4e-01 / 1.1e+00 | rank:  1.0 | time:   0.265
624972500385.0 0.0
Rastrigin       [CRO          ] > error: 2.5e-02 / 2.4e-01 | rank:  1.0 | time:   0.222
398.00479787978617 0.0
Rosenbrock      [CRO          ] > error: 9.7e-02 / 1.1e+00 | rank:  1.0 | time:   0.222
35153.336041574395 0.0
Schaffer      

---